<a href="https://colab.research.google.com/github/Sanaghufan/Computer-Vision-and-Deep-Learning-Projects/blob/main/SkinCancerDenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["KAGGLE_USERNAME"] ="sana206"
os.environ["KAGGLE_KEY"] = "7aa7b4eb5c12f3b5324507ef68ac0dc8"
!kaggle datasets download nodoubttome/skin-cancer9-classesisic

skin-cancer9-classesisic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import cv2
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from IPython.display import clear_output
!unzip skin-cancer9-classesisic.zip
clear_output()

In [ ]:
train_path = "dataset/Train"
test_path = "dataset/Test"

In [ ]:
class_names = os.listdir(train_path)
class_names

['pigmented benign keratosis',
 'melanoma',
 'actinic keratosis',
 'nevus',
 'basal cell carcinoma',
 'squamous cell carcinoma',
 'dermatofibroma',
 'seborrheic keratosis',
 'vascular lesion']

In [ ]:
# Pre-processing of images
image_data_gen = ImageDataGenerator(horizontal_flip=True,
                                    rescale=1/255.

                                    )

In [ ]:
X_train = image_data_gen.flow_from_directory(train_path, target_size=(224,224), batch_size=32, class_mode="sparse")

Found 2239 images belonging to 9 classes.


In [ ]:
X_val = image_data_gen.flow_from_directory(test_path, target_size=(224,224), batch_size=32, class_mode="sparse")

Found 118 images belonging to 9 classes.


In [ ]:
'''x_train_mean = np.mean(X_train)
x_train_std = np.std(X_train)
x_test_mean = np.mean(X_val)
x_test_std = np.std(X_val)

X_train = (X_train - x_train_mean)/x_train_std
X_val = (X_val - x_test_mean)/x_test_std'''

'x_train_mean = np.mean(X_train)\nx_train_std = np.std(X_train)\nx_test_mean = np.mean(X_val)\nx_test_std = np.std(X_val)\n\nX_train = (X_train - x_train_mean)/x_train_std\nX_val = (X_val - x_test_mean)/x_test_std'

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# DenseNet201
model = Sequential()
model.add(DenseNet201(include_top=False, weights='imagenet', input_shape=(224,224,3)))
model.add(Flatten())
model.add(Dropout(0.5))  # Add a Dropout layer with a dropout rate of 0.5
model.add(Dense(512, activation='relu'))
model.add(Dense(9, activation='softmax'))
model.summary()

74836368/74836368 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 flatten (Flatten)           (None, 94080)             0         
                                                                 
 dropout (Dropout)           (None, 94080)             0         
                                                                 
 dense (Dense)               (None, 512)               48169472  
                                                                 
 dense_1 (Dense)             (None, 9)                 4617      
                                                                 
Total params: 66,496,073
Trainable params: 66,267,017
Non-trainable params: 229,056
_______________________________________

In [ ]:
# compile model
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])


# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.05,
                                            min_lr=0.00001)

In [ ]:
# Fit the model
epochs = 100
batch_size=32
history = model.fit(X_train,epochs=epochs,batch_size=batch_size,validation_data=X_val,callbacks=learning_rate_reduction)

Epoch 1/100
70/70 [==============================] - ETA: 0s - loss: 1.6841 - accuracy: 0.4855

70/70 [==============================] - 124s 1s/step - loss: 1.6841 - accuracy: 0.4855 - val_loss: 2.4951 - val_accuracy: 0.2288 - lr: 0.0010
Epoch 2/100
70/70 [==============================] - ETA: 0s - loss: 0.8168 - accuracy: 0.7124

70/70 [==============================] - 63s 889ms/step - loss: 0.8168 - accuracy: 0.7124 - val_loss: 2.6069 - val_accuracy: 0.3051 - lr: 0.0010
Epoch 3/100
70/70 [==============================] - ETA: 0s - loss: 0.6254 - accuracy: 0.7686

70/70 [==============================] - 63s 898ms/step - loss: 0.6254 - accuracy: 0.7686 - val_loss: 2.8807 - val_accuracy: 0.3644 - lr: 0.0010
Epoch 4/100
70/70 [==============================] - ETA: 0s - loss: 0.4469 - accuracy: 0.8361

70/70 [==============================] - 63s 899ms/step - loss: 0.4469 - accuracy: 0.8361 - val_loss: 3.6385 - val_accuracy: 0.3644 - lr: 0.0010
Epoch 5/100
70/70 [==============================] - ETA: 0s - loss: 0.3722 - accuracy: 0.8571

...

70/70 [==============================] - 61s 868ms/step - loss: 0.0898 - accuracy: 0.9366 - val_loss: 5.7874 - val_accuracy: 0.4915 - lr: 0.0010
Epoch 97/100
70/70 [==============================] - ETA: 0s - loss: 0.0877 - accuracy: 0.9361

70/70 [==============================] - 62s 877ms/step - loss: 0.0877 - accuracy: 0.9361 - val_loss: 5.6676 - val_accuracy: 0.5085 - lr: 0.0010
Epoch 98/100
70/70 [==============================] - ETA: 0s - loss: 0.0915 - accuracy: 0.9241

70/70 [==============================] - 61s 872ms/step - loss: 0.0915 - accuracy: 0.9241 - val_loss: 5.0189 - val_accuracy: 0.4746 - lr: 0.0010
Epoch 99/100
70/70 [==============================] - ETA: 0s - loss: 0.0908 - accuracy: 0.9357

70/70 [==============================] - 61s 867ms/step - loss: 0.0908 - accuracy: 0.9357 - val_loss: 5.1189 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 100/100
70/70 [==============================] - ETA: 0s - loss: 0.0907 - accuracy: 0.9348

70/70 [==============================] - 61s 867ms/step - loss: 0.0907 - accuracy: 0.9348 - val_loss: 6.7861 - val_accuracy: 0.4831 - lr: 0.0010


In [ ]:

loss, accuracy = model.evaluate(X_train,verbose=1)
print("Train: accuracy = %f  ;  loss = %f" % (accuracy, loss))

70/70 [==============================] - 42s 597ms/step - loss: 0.0888 - accuracy: 0.9393
Train: accuracy = 0.939259  ;  loss = 0.088770


In [ ]:
loss, accuracy = model.evaluate(X_val, verbose=1)
print("Testing: accuracy = %f  ;  loss = %f" % (accuracy, loss))

4/4 [==============================] - 10s 2s/step - loss: 6.6305 - accuracy: 0.4915
Testing: accuracy = 0.491525  ;  loss = 6.630476
